# RAG On JFK Speeches: Part 2

### 1. Introduction

In [5]:
# LangChain
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain_pinecone import PineconeVectorStore
from langchain.chains.retrieval import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain import hub

# Pinecone VectorDB
from pinecone import Pinecone
from pinecone import ServerlessSpec

import os

# API Keys
from dotenv import load_dotenv
load_dotenv()


True

## 2. Retriving Documents With Contextual Search

Create the connection and list out the indices

In [ ]:
index_name = "prez-speeches"

pc = Pinecone(api_key=os.getenv("PINECONE_API_KEY"))
pc.list_indexes()

[
    {
        "name": "prez-speeches",
        "dimension": 1536,
        "metric": "cosine",
        "host": "prez-speeches-2307pwa.svc.aped-4627-b74a.pinecone.io",
        "spec": {
            "serverless": {
                "cloud": "aws",
                "region": "us-east-1"
            }
        },
        "status": {
            "ready": true,
            "state": "Ready"
        },
        "deletion_protection": "disabled"
    }
]

Instantiate the embeddings

In [8]:
embedding = OpenAIEmbeddings(model='text-embedding-ada-002')


Create the initial connection to the Vector database:

In [9]:
vectordb = PineconeVectorStore(
                    pinecone_api_key=os.getenv("PINECONE_API_KEY"),
                    embedding=embedding,
                    index_name=index_name
)

In [72]:
index = pc.Index(index_name)

In [73]:
question = "How did Kennedy feel about the Berlin Wall?"

In [74]:
matches = index.query(vector=query, top_k=5)

In [76]:
matches

{'matches': [{'id': 'a48ee926-4c6c-4614-aebf-bc8ea77d9cd3',
              'score': 0.839695573,
              'values': []},
             {'id': '3150a237-c987-4332-adf3-92282f838222',
              'score': 0.839501321,
              'values': []},
             {'id': '2fd52eb5-397a-42f1-a8f3-f2e04471dc20',
              'score': 0.835366726,
              'values': []},
             {'id': '05385d3c-bf94-4f70-84e0-54b56254915e',
              'score': 0.829040587,
              'values': []},
             {'id': 'f3f7962b-790e-402a-9827-0234de0b28ef',
              'score': 0.827951968,
              'values': []}],
 'namespace': '',
 'usage': {'read_units': 5}}

In [77]:
id = matches["matches"][0].get('id')

In [78]:
id

'a48ee926-4c6c-4614-aebf-bc8ea77d9cd3'

In [79]:
index.fetch(id)

{'namespace': '', 'usage': {'read_units': 4}, 'vectors': {}}

In [80]:
results = vectordb.search(query=question, search_type="similarity")

In [81]:
for doc in results:
    print()

[Document(id='05385d3c-bf94-4f70-84e0-54b56254915e', metadata={'filename': 'berlin-crisis-19610725', 'seq_num': 1.0, 'source': 'gs://prezkennedyspeches/berlin-crisis-19610725.json', 'title': 'Radio and Television Report to the American People on the Berlin Crisis, July 25, 1961', 'url': 'https://www.jfklibrary.org//archives/other-resources/john-f-kennedy-speeches/berlin-crisis-19610725'}, page_content='Listen to the speech. \xa0\xa0 View related documents. \nPresident John F. Kennedy\nThe White House\nJuly 25, 1961\nGood evening:\nSeven weeks ago tonight I returned from Europe to report on my meeting with Premier Khrushchev and the others. His grim warnings about the future of the world, his aide memoire on Berlin, his subsequent speeches and threats which he and his agents have launched, and the increase in the Soviet military budget that he has announced, have all prompted a series of decisions by the Administration and a series of consultations with the members of the NATO organizat

## 3. Building A RAG Pipeline

In [181]:
template = hub.pull("langchain-ai/retrieval-qa-chat")

In [212]:
print(template.messages[0].prompt)

input_variables=['context'] input_types={} partial_variables={} template='Answer any use questions based solely on the context below:\n\n<context>\n{context}\n</context>'


In [214]:
print(template.messages[1])

variable_name='chat_history' optional=True


In [211]:
print(template.messages[2].prompt)

input_variables=['input'] input_types={} partial_variables={} template='{input}'


In [215]:
print(template.input_variables)

['context', 'input']


In [182]:
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)

In [183]:
stuff_documents_chain = create_stuff_documents_chain(llm, template)

In [184]:
rag_chain = create_retrieval_chain(
                    vectordb.as_retriever(), 
                    stuff_documents_chain
)

In [217]:
rag_chain.get_prompts()

[PromptTemplate(input_variables=['page_content'], input_types={}, partial_variables={}, template='{page_content}'),
 ChatPromptTemplate(input_variables=['context', 'input'], optional_variables=['chat_history'], input_types={'chat_history': list[typing.Annotated[typing.Union[typing.Annotated[langchain_core.messages.ai.AIMessage, Tag(tag='ai')], typing.Annotated[langchain_core.messages.human.HumanMessage, Tag(tag='human')], typing.Annotated[langchain_core.messages.chat.ChatMessage, Tag(tag='chat')], typing.Annotated[langchain_core.messages.system.SystemMessage, Tag(tag='system')], typing.Annotated[langchain_core.messages.function.FunctionMessage, Tag(tag='function')], typing.Annotated[langchain_core.messages.tool.ToolMessage, Tag(tag='tool')], typing.Annotated[langchain_core.messages.ai.AIMessageChunk, Tag(tag='AIMessageChunk')], typing.Annotated[langchain_core.messages.human.HumanMessageChunk, Tag(tag='HumanMessageChunk')], typing.Annotated[langchain_core.messages.chat.ChatMessageChunk,

In [218]:
response = rag_chain.invoke({"input": question})

In [219]:
print(response["answer"])

Kennedy viewed the Berlin Wall as a vivid demonstration of the failures of the Communist system and an offense against humanity. He expressed no satisfaction in its existence, recognizing it as a barrier that separated families and divided people who wished to be united. He emphasized the vitality, hope, and determination of the people of West Berlin despite the wall's presence.


In [234]:
response

{'input': 'How did Kennedy feel about the Berlin Wall?',
 'context': [Document(id='6b164251-8eec-4fd8-8ca9-00b94671d288', metadata={'filename': 'berlin-w-germany-rudolph-wilde-platz-19630626', 'seq_num': 1.0, 'source': 'gs://prezkennedyspeches/berlin-w-germany-rudolph-wilde-platz-19630626.json', 'title': 'Remarks of President John F. Kennedy at the Rudolph Wilde Platz, Berlin, June 26, 1963', 'url': 'https://www.jfklibrary.org//archives/other-resources/john-f-kennedy-speeches/berlin-w-germany-rudolph-wilde-platz-19630626'}, page_content='Freedom has many difficulties and democracy is not perfect, but we have never had to put a wall up to keep our people in, to prevent them from leaving us. I want to say, on behalf of my countrymen, who live many miles away on the other side of the Atlantic, who are far distant from you, that they take the greatest pride that they have been able to share with you, even from a distance, the story of the last 18 years. I know of no town, no city, that has

In [82]:
references = [(doc.metadata["title"], doc.page_content, doc.metadata["url"]) 
              for doc in response['context']]

NameError: name 'response' is not defined

In [83]:
references

NameError: name 'references' is not defined

## 4. Deploying A RAG App